In [ ]:
import numpy as np
import pandas as pd
import yaml

with open("../env_vars.yml", "r") as file:
    config = yaml.safe_load(file)

import warnings

warnings.filterwarnings("ignore")

# set formatting
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)

# Clean raw data

In [ ]:
# NOTE: Read static data
df_raw = pd.read_csv(f"{config['data_directory']}/data_showcase.csv", sep="\t")

In [3]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90035 entries, 0 to 90034
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Product Area             90035 non-null  object 
 1   Core Segment             90035 non-null  object 
 2   Brand                    90035 non-null  object 
 3   Material Number          90035 non-null  int64  
 4   Material No Text         90035 non-null  object 
 5   Component                90035 non-null  object 
 6   Material Description     90035 non-null  object 
 7   Packaging Code           90035 non-null  object 
 8   Material Characteristic  90035 non-null  object 
 9   Material Weight          82545 non-null  float64
 10  Column 21                90035 non-null  float64
 11  Weight measure           82545 non-null  object 
 12  Packaging Category       88770 non-null  object 
 13  Manufactoring Location   90035 non-null  object 
 14  Column 43             

In [4]:
df_raw.describe()

,Material Number,Material Weight,Column 21,Column 43
count,9.003500e+04,8.254500e+04,90035.000000,8.567100e+04
mean,5.509347e+07,4.581777e+02,2.674080,4.941166e+04
std,2.591986e+07,2.995114e+04,15.922722,2.016637e+05
min,1.000228e+07,0.000000e+00,0.000000,-3.014549e+04
25%,3.268354e+07,8.100000e-01,0.000000,1.722010e+03
50%,5.509066e+07,5.000000e+00,0.000000,7.460691e+03
75%,7.733171e+07,2.570000e+01,0.000000,2.997832e+04
max,9.999433e+07,2.300000e+06,328.000000,1.075813e+07


## Rename columns for easier handling

In [ ]:
df_cleaned = df_raw.rename(
    columns={
        "Product Area": "product_area",
        "Core Segment": "core_segment",
        "Brand": "brand",
        "Material Number": "material_number",
        "Material No Text": "material_number_text",
        "Component": "component",
        "Material Description": "component_text",
        "Packaging Code": "packaging_code",
        "Material Characteristic": "characteristic_value",
        "Material Weight": "material_weight",
        "Column 21": "col_21",
        "Weight measure": "weight_measure",
        "Packaging Category": "packaging_category",
        "Manufactoring Location": "manufactoring_location",
        "Column 43": "col_43",
    }
)

## Clean "packaging_category"

In [ ]:
# clean rows
df_cleaned["packaging_category"].mask(
    df_cleaned["packaging_category"].isin(["-", np.nan]), "Unassigned", inplace=True
)
df_cleaned["packaging_category"].mask(
    df_cleaned["packaging_category"].isin(["No Packaging"]), "Unpacked", inplace=True
)

## Create subdataset with only the features we want to use

In [ ]:
#  sort and filter columns
df_sub = df_cleaned[
    [
        "material_number",
        "material_number_text",
        "brand",
        "product_area",
        "core_segment",
        "component",
        "component_text",
        "manufactoring_location",
        "characteristic_value",
        "material_weight",
        # 'weight_measure', # ignore since most values are 'G', 'NaN' might be an issue
        "packaging_code",
        "packaging_category",
        # 'col_21'
        # 'col_43'
    ]
]

In [13]:
df_sub.head()

,material_number,material_number_text,brand,product_area,core_segment,component,component_text,manufactoring_location,characteristic_value,material_weight,packaging_code,packaging_category
0,75116293,Counter Display,BOT,PA5,Metal Grinding,6035765C21,Corrugated carton,Distribution Center,CORRUGATED,85.00,PCode_304109,Countertop display
1,75116293,Counter Display,BOT,PA5,Metal Grinding,6035940565,Label SB,Distribution Center,WOOD FREE,0.54,PCode_440854,Countertop display
2,75116293,Counter Display,BOT,PA5,Metal Grinding,6035822768,Tight -Pack label RB - 1ER,Distribution Center,MCB/GT2,22.90,PCode_834649,Countertop display
3,75116293,Counter Display,BOT,PA5,Metal Grinding,6035822768,Tight -Pack label RB - 1ER,Distribution Center,MCB/GT2,22.90,PCode_834649,Countertop display
4,75116293,Counter Display,BOT,PA5,Metal Grinding,6035765P54,Corrugated carton,Distribution Center,CORRUGATED,85.00,PCode_304109,Countertop display


# Split data into 1. data with packaging categories for machine learning workflow, 2. data without packaging categories to do predictions for.

In [ ]:
# Split data into 1. data with packaging categories for machine learning workflow, 2. data without packaging categories to do predictions for.
df_ml = df_sub[df_sub.packaging_category != "Unassigned"]
df_no_packaging_categories = df_sub[df_sub.packaging_category == "Unassigned"]

print(f"{df_ml.shape} df_ml shape, data with packaging categories")
print(
    f"{df_no_packaging_categories.shape} df_no_packaging_categories shape, data without packaging categories"
)

## Save processed data: 1. data set for machine learning workflow, 2. data to be predicted

In [ ]:
df_ml.to_csv(f"{config['data_directory']}/output/df_ml.csv", sep="\t", index=False, header=True)

In [ ]:
df_no_packaging_categories.to_csv(
    f"{config['data_directory']}/output/df_inf.csv", sep="\t", index=False, header=True
)